In [ ]:
!git init
!git config --global user.email "wheelessbrian@yahoo.com"
!git config --global user.name "bwheeless7"
!git remote add origin https://github.com/bwheeless7/data-portfolio.git


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!mkdir -p churn-retention-analysis/notebooks
!mkdir -p churn-retention-analysis/data
!mkdir -p churn-retention-analysis/src


In [ ]:
!mv "/content/01_problem_definition_and_eda.ipynb" churn-retention-analysis/notebooks/


mv: cannot stat '/content/01_problem_definition_and_eda.ipynb': No such file or directory


In [ ]:
!find /content -maxdepth 2 -type f

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/data-portfolio/churn-retention-analysis
!ls -a

/content/drive/MyDrive/data-portfolio/churn-retention-analysis
data  notebooks  README.md  src


In [ ]:
!git remote -v


In [11]:
!git config --global user.name "bwheeless7"
!git config --global user.email "wheelessbrian@yahoo.com"


In [ ]:
!git status
# !git add /content/drive/MyDrive/data-portfolio/churn-retention-analysis/notebooks/01_problem_definition_and_eda.ipynb
# !git commit -m "Progress on problem definition and EDA"



In [ ]:
!git pull origin main --no-rebase


From https://github.com/bwheeless7/data-portfolio
 * branch            main       -> FETCH_HEAD
error: Your local changes to the following files would be overwritten by merge:
	churn-retention-analysis/notebooks/01_problem_definition_and_eda.ipynb
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
!git add .
!git commit -m "Update notebook with new analysis"
!git push

In [ ]:
# New Workflow
# !git pull

!git add .
!git commit -m "Train baseline churn model with strong ROC-AUC and recall"
e

# !git push


Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (7/7), 762 bytes | 12.00 KiB/s, done.
Total 7 (delta 3), reused 3 (delta 3), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
To https://github.com/bwheeless7/data-portfolio.git
 + 3e89389...7cf34c9 main -> main (forced update)


# Business Problem
A digital bank is experiencing increased customer attrition, which directly impacts revenue and long-term growth.
The objective is to identify the key drivers of customer churn, build a predictive model, and propose retention strategies.

# Success Metrics
- Churn Rate
- Retention Rate
- Model ROC-AUC
- Precision/Recall at business-selected threshold


In [4]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/data-portfolio/churn-retention-analysis/data/BankChurners.csv")


In [5]:
df.shape
df['Attrition_Flag'].value_counts(normalize=True)

,proportion
Attrition_Flag,
Existing Customer,0.83934
Attrited Customer,0.16066


## Initial Findings

- Overall churn rate is approximately **16.1%**, which is significant for a financial services business.
- The dataset is moderately imbalanced, so evaluation will emphasize recall, precision, and ROC-AUC over raw accuracy.
- Even small improvements in churn reduction could yield substantial revenue impact.


In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [7]:
df = df.drop(columns=[
    'CLIENTNUM',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
])


## Data Preparation Notes

Several columns were removed prior to modeling:

- `CLIENTNUM` (unique identifier)
- Two Naive Bayes probability columns that directly leak the target variable

These features would artificially inflate model performance and are excluded to preserve model integrity.


In [8]:
df['Attrition_Flag'] = df['Attrition_Flag'].map({
    'Existing Customer': 0,
    'Attrited Customer': 1
})

X = df.drop('Attrition_Flag', axis=1)
y = df['Attrition_Flag']

X.shape, y.mean()


((10127, 19), np.float64(0.1606596227905599))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


((8101, 19),
 (2026, 19),
 np.float64(0.1607208986544871),
 np.float64(0.16041461006910168))

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

categorical_cols = X.select_dtypes(include='object').columns
numeric_cols = X.select_dtypes(exclude='object').columns

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred))


ROC-AUC: 0.9206765251209696
              precision    recall  f1-score   support

           0       0.96      0.86      0.91      1701
           1       0.53      0.82      0.64       325

    accuracy                           0.85      2026
   macro avg       0.75      0.84      0.78      2026
weighted avg       0.89      0.85      0.87      2026



## Baseline Model Performance

The baseline logistic regression model achieved:

- **ROC-AUC: 0.921**, indicating excellent overall discrimination.
- **Churn Recall: 0.82**, meaning the model identifies 82% of customers who will churn.
- **Churn Precision: 0.53**, reflecting the tradeoff between recall and false positives.
- **Overall Accuracy: 85%**, though accuracy is secondary due to class imbalance.

From a business perspective, the model is highly effective at capturing potential churners, enabling proactive retention strategies.
